# Query that returns both text (name) and code (ID) from JSON-stat
### Example: HS codes for foreign trade

### Import libraries
Uses [pyjstat](https://pypi.org/project/pyjstat/) and pandas.


In [1]:
from pyjstat import pyjstat  
import pandas as pd  # Pandas is actually loaded as part of pyjstat
import requests

Import/Export all product codes to US for the last month, approx. 65,000 cells. Max limit for a query in PxWebApi is 800,000 cells, including empty cells.


### URL with the table's metadata


GET_URL = https://data.ssb.no/api/pxwebapi/v2/tables/08801/data?lang=en&valueCodes[Varekoder]=*&valueCodes[ImpEks]=1,2&valueCodes[ContentsCode]=Mengde1,Verdi,Mengde2&valueCodes[Tid]=2024&codelist[Varekoder]=vs_VareKoderKN8Siff&codelist[Land]=vs_LandKoderAlf1

JSON-stat2 is the default output format. You can also add others with `&outputFormat=` such as csv or xlsx.

Note that the API URLs are *not* case-sensitive.

Showing the individual elements. The codelist elements are in most cases superfluous/redundant and can be removed.


In [13]:
# The individual elements in the query

tabid = "08799" # can be changed to the annual table 08801, without other changes
lang = "lang=en" # language code can also be "en"
get_stem = "https://data.ssb.no/api/pxwebapi/v2/tables/" + tabid + "/data?" + lang   # 'https://data.ssb.no/api/v0/no/table/08799'
get_query = "&valueCodes[Varekoder]=*\
&valueCodes[ImpEks]=1,2\
&valueCodes[Land]=US\
&valueCodes[ContentsCode]=Mengde1,Verdi,Mengde2\
&valueCodes[Tid]=top(1)"

Import/Export all product codes to US for the last month, approx. 65,000 cells. Max limit for a query in PxWebApi is 800,000 cells, including empty cells.


In [14]:
GET_URL = get_stem + get_query 

In [15]:
res = requests.get(GET_URL)

HTTP code 200 is OK


In [16]:
res

<Response [200]>

### Read the JSON-stat result using the pyjstat library
Store it as dataset ds.


In [17]:
ds = pyjstat.Dataset.read(res.text)

In [18]:
type(ds)

pyjstat.pyjstat.Dataset

View the dataset ds - big dataset


In [19]:
# ds

### Fetch some top-level metadata from the JSON-stat dataset


In [20]:
tittel = ds['label']
print(tittel)

08799: External trade in goods, by commodity number, imports/exports, country, contents and month


Last Updated as GMT


In [21]:
last_update = ds['updated']
print(last_update)

2025-10-15T06:00:00Z


Fetch source


In [22]:
source = ds['source']
print(source)

Statistics Norway


Roles provide some shortcuts to the IDs of fixed dimensions in the dataset


In [23]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'geo': ['Land'], 'metric': ['ContentsCode']})


### Create dataframes
We need to create two dataframes, one with text and one with ID. Pyjstat only allows either/or


In [24]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [25]:
hstrade.head()

,commodity number,imports/exports,country,contents,month,value
0,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Quantity 1 (Q1),2025M09,0
1,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Value (NOK),2025M09,0
2,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Quantity 2 (Q2),2025M09,0
3,"Imports of unspecified goods, with duty (1988-...",Exports,United States of America,Quantity 1 (Q1),2025M09,0
4,"Imports of unspecified goods, with duty (1988-...",Exports,United States of America,Value (NOK),2025M09,0


In [26]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,00000011_1988,1,US,Mengde1,2025M09,0
1,00000011_1988,1,US,Verdi,2025M09,0
2,00000011_1988,1,US,Mengde2,2025M09,0
3,00000011_1988,2,US,Mengde1,2025M09,0
4,00000011_1988,2,US,Verdi,2025M09,0


### Create a new column with both HS product code and text combined


In [28]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['commodity number']

In [29]:
hstrade.columns

Index(['commodity number', 'imports/exports', 'country', 'contents', 'month',
       'value', 'hstrade_combi'],
      dtype='object')

### Create a new dataframe with only the columns we want, in new order.
Note double [[ ]] to create dataframe


In [32]:
hstrade_new = hstrade[['hstrade_combi', 'imports/exports', 'country', 'contents', 'month',
       'value']]

In [33]:
hstrade_new.tail(10)

,hstrade_combi,imports/exports,country,contents,month,value
80408,"99603000_2023 VOEC-shipments, goods with value...",Imports,United States of America,Quantity 2 (Q2),2025M09,0
80409,"99603000_2023 VOEC-shipments, goods with value...",Exports,United States of America,Quantity 1 (Q1),2025M09,0
80410,"99603000_2023 VOEC-shipments, goods with value...",Exports,United States of America,Value (NOK),2025M09,0
80411,"99603000_2023 VOEC-shipments, goods with value...",Exports,United States of America,Quantity 2 (Q2),2025M09,0
80412,"99999999_1988 Total, all commodities not for p...",Imports,United States of America,Quantity 1 (Q1),2025M09,8592609
80413,"99999999_1988 Total, all commodities not for p...",Imports,United States of America,Value (NOK),2025M09,34056104
80414,"99999999_1988 Total, all commodities not for p...",Imports,United States of America,Quantity 2 (Q2),2025M09,0
80415,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Quantity 1 (Q1),2025M09,68405885
80416,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Value (NOK),2025M09,482185095
80417,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Quantity 2 (Q2),2025M09,0


Change column width.


In [34]:
pd.set_option('display.max_colwidth',140)

In [35]:
hstrade_new.tail(10)

,hstrade_combi,imports/exports,country,contents,month,value
80408,"99603000_2023 VOEC-shipments, goods with value under NOK 3 000, natural person (2023-) (Q1=kg, Q2=none)",Imports,United States of America,Quantity 2 (Q2),2025M09,0
80409,"99603000_2023 VOEC-shipments, goods with value under NOK 3 000, natural person (2023-) (Q1=kg, Q2=none)",Exports,United States of America,Quantity 1 (Q1),2025M09,0
80410,"99603000_2023 VOEC-shipments, goods with value under NOK 3 000, natural person (2023-) (Q1=kg, Q2=none)",Exports,United States of America,Value (NOK),2025M09,0
80411,"99603000_2023 VOEC-shipments, goods with value under NOK 3 000, natural person (2023-) (Q1=kg, Q2=none)",Exports,United States of America,Quantity 2 (Q2),2025M09,0
80412,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Imports,United States of America,Quantity 1 (Q1),2025M09,8592609
80413,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Imports,United States of America,Value (NOK),2025M09,34056104
80414,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Imports,United States of America,Quantity 2 (Q2),2025M09,0
80415,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Exports,United States of America,Quantity 1 (Q1),2025M09,68405885
80416,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Exports,United States of America,Value (NOK),2025M09,482185095
80417,"99999999_1988 Total, all commodities not for publication (1988-) (Q1=kg, Q2=none)",Exports,United States of America,Quantity 2 (Q2),2025M09,0
